In [17]:
%%time
# core importations
from caveclient import CAVEclient
import pandas as pd

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

# ----------CORE DATA GATHERING FUCNTIONS----------

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to get nucleus table entry using root id
def getNuc(root):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": [root]},
                                            materialization_version = mat_vers)
    return nuc_df 

# defines function to get synapse table entry using root id
def getSyn(root):
    client = CAVEclient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root]})
    return len(syn_df)

# defines function to get edit data
def getEdits(root):
    client = CAVEclient("flywire_fafb_production")
    changes_dict = client.chunkedgraph.get_change_log(root)
    return changes_dict   

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilder(root_list,options):
    data_list = []
    column_list = ['root_id']
    if 'nucs' in options:
        nuc = True
        column_list += ['nucleus_id','nucleus_coordinates','volume']
    else:
        nuc = False
    if 'syns' in options:
        syn = True
        column_list += ['synapses']
    else:
        syn = False
    if 'edits' in options:
        edits = True
        column_list += ['mergers','splits','edits']
    else:
        edits = False     
    for i in root_list:
        sub_list = []
        sub_list.append(i) #adds root#
        if nuc == True:
            nuc_df = getNuc(i) #pulls nucleus table#
            sub_list.append(nuc_df.loc[0,'id']) #adds nucleus id #
            sub_list.append(coordConvert(nuc_df.loc[0,'pt_position'])) #adds coordinates#
            sub_list.append(nuc_df.loc[0,'volume']) #adds volume#
        if syn == True:
            sub_list.append(getSyn(i)) #adds synapses#
        if edits == True:
            change_dict = getEdits(i) #pulls changelog#
            sub_list.append(change_dict['n_splits']) #adds splits# 
            sub_list.append(change_dict['n_mergers']) #adds mergers#
            sub_list.append((sub_list[-1] + sub_list[-2])) #adds edits#
#             sub_list.append(change_dict['user_info']) #adds editors, TEMPORARILY DISABLED#
        data_list.append(sub_list) #adds entry to data_list#

    # converts data_list into dataframe
    df = pd.DataFrame(data_list,columns = column_list)
    return df
   

Wall time: 0 ns


In [20]:
# DASH APP

app = dash.Dash(__name__)

# defines layout of various app elements (submission field, checkboxes, button, output table)#
app.layout = html.Div([
    html.Div(dcc.Input(  #defines input field#
        id='input_field', 
        type='text', 
        placeholder='Root ID(s)',
    )),
    html.Br(
    ),
    html.Div(dcc.Checklist(  #defines data selection checkboxes#
        id='checkboxes',
        options=[
            {'label': 'Nucleus Info', 'value': 'nucs'},
            {'label': 'Synapse Count', 'value': 'syns'},
            {'label': 'Edits', 'value': 'edits'},
        ],
        labelStyle={'display': 'block'},
        value=['nucs','syns','edits'],
    )),
    html.Br(
    ),
    html.Button(  #defines submission button#
        'Submit', 
        id='submit_button', 
        n_clicks=0,
    ),
    html.Br(
    ),
    html.Div(dash_table.DataTable(  #defines output table#
        id='table', 
        fill_width=False,  #sets column width to fit text instead of expanding to container width#
    ))
])

# defines callback that takes root ids and desired data selection on button click and generates table
@app.callback(
    Output('table','columns'),           #defines first output location as the 'columns' aspect of 'table'#
    Output('table', 'data'),             #defines second output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'),  #defines trigger as button press (change in the state of the 'n_clicks' aspect of 'submit_button')# 
    State('input_field', 'value'),       #defines first input state as the value of 'input_field'#
    State('checkboxes','value')          #defines second input state as the value of 'checkboxes'#
)
def update_output(n_clicks, roots, checked):
    root_list = str(roots).split(",")                         #splits 'roots' string into list#
    root_list = [int(x.strip(' ')) for x in root_list]        #strips spaces from root_list entries and converts to integers#
    df = dfBuilder(root_list, checked)                        #feeds root_list and checkbox values into dataframe builder#
    column_list = [{"name": i, "id": i} for i in df.columns]  #creates column list based on dataframe columns#
    df_dict =  df.to_dict('records')                          #converts df to dictionary#
    output_list = [column_list,df_dict]                       #combines df_dict and column_list into output list#
    return output_list                                        #returns list of column names and data values#

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2021 16:59:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 16:59:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 16:59:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 16:59:18] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 16:59:18] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 16:59:18] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_use

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_use

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_use

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_use

127.0.0.1 - - [22/Oct/2021 16:59:21] "POST /_dash-update-component HTTP/1.1" 500 -


{'source': 'precomputed://https://storage.googleapis.com/neuroglancer/basil_v0/son_of_alignment/v3.04_cracks_only_normalized_rechunked',
 'type': 'image',
 'blend': 'default',
 'shaderControls': {},
 'name': 'img'}